In [1]:
import gspread
import pandas as pd
import numpy as np

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [39]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]
df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [41]:
df['date'].unique()

array(['12/27/2022', '12/28/2022', '12/29/2022', '12/30/2022', '1/3/2023',
       '1/4/2023', '1/5/2023', '1/6/2023', '1/7/2023', '1/10/2023',
       '1/12/2023', '1/14/2023', '1/17/2023', '1/20/2023', '1/21/2023',
       '1/22/2023', '1/25/2023', '1/28/2023', '1/29/2023', '1/31/2023',
       '2/3/2023', '2/5/2023', '2/7/2023', '2/11/2023', '2/17/2023',
       '2/22/2023', '2/23/2023', '2/25/2023', '3/2/2023', '12/24/2022',
       '12/26/2022'], dtype=object)

In [57]:
players = []
game_date = '1/28/2023'

for i in [1, 2]:
    players += df[df['date'] == game_date][f'team_{i}_player_1'].to_list() + df[df['date'] == game_date][f'team_{i}_player_2'].to_list()
    
set(players), df[df['date'] == game_date]['venue'].unique()

({'aakarsh', 'ajay', 'prateek', 'raghotham', 'vinay'},
 array(['match point - gublaala'], dtype=object))

In [59]:
set().remove()

Help on built-in function union:

union(...) method of builtins.set instance
    Return the union of sets as a new set.
    
    (i.e. all elements that are in either set.)

